In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import keras_tuner as kt
import multiprocessing

from tensorflow import data
from tensorflow.keras import Sequential, Input, Model
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Softmax
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy
from tensorflow.keras.applications import resnet50
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
from fl_tissue_model_tools import data_prep, models

In [3]:
n_cores = multiprocessing.cpu_count()
n_cores

16

In [4]:
root_data_path = f"D:/malaria_data/cell_images_split"
seed = 2049
# Resize using "area" method
# Multiples of 32 work best with ResNet
resnet_inp_shape = (128, 128, 3)
n_outputs = 1
val_split = 0.2
batch_size = 32
frozen_epochs = 10
fine_tune_epochs = 10
adam_beta_1_range = (0.85, 0.95)
adam_beta_2_range = (0.98, 0.999)
frozen_lr_range = (1e-4, 1e-2)
fine_tune_lr_range = (1e-5, 1e-3)
last_layer_options = ["conv5_block3_out", "conv5_block2_out", "conv5_block1_out", "conv4_block6_out"]
# 5 hyperparamters, 3 * 5 is opt default, 3 times as many as default
max_opt_trials = 3 * (3 * 5)
project_name = "malaria_hp_trials"

# Early stopping
es_criterion = "val_loss"
es_mode = "min"
# Change this for serious experiments
es_patience = 1
es_min_delta = 0.1

# Data

## Data split

In [5]:
train_datagen = ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=360,
    # Make sure to preprocess the same as the original model
    preprocessing_function=resnet50.preprocess_input,
    # Use 20% of data for validation
    validation_split=val_split
)

In [6]:
test_datagen = ImageDataGenerator(
    preprocessing_function=resnet50.preprocess_input
)

## Generators

In [7]:
train_generator = train_datagen.flow_from_directory(
    f"{root_data_path}/train",
    target_size=resnet_inp_shape[:2],
    batch_size=batch_size,
    # Want uninfected to be labeled "0"
    classes={"uninfected": 0, "parasitized": 1},
    class_mode="binary",
    seed=seed,
    subset="training"
)

val_generator = train_datagen.flow_from_directory(
    f"{root_data_path}/train",
    target_size=resnet_inp_shape[:2],
    batch_size=batch_size,
    classes={"uninfected": 0, "parasitized": 1},
    class_mode="binary",
    seed=seed,
    subset="validation"
)

Found 17638 images belonging to 2 classes.
Found 4408 images belonging to 2 classes.


In [8]:
test_generator = test_datagen.flow_from_directory(
    f"{root_data_path}/test",
    target_size=resnet_inp_shape[:2],
    batch_size=batch_size,
    classes={"uninfected": 0, "parasitized": 1},
    class_mode=None,
    shuffle=False
)

Found 5512 images belonging to 2 classes.


# Build hyper model

In [9]:
hypermodel = models.ResNet50TLHyperModel(
    n_outputs=n_outputs,
    img_shape=resnet_inp_shape,
    loss=BinaryCrossentropy(),
    metrics=[BinaryAccuracy()],
    name="malaria_hypermodel",
    output_act="sigmoid",
    adam_beta_1_range=adam_beta_1_range,
    adam_beta_2_range=adam_beta_2_range,
    frozen_lr_range=frozen_lr_range,
    fine_tune_lr_range=fine_tune_lr_range,
    frozen_epochs=frozen_epochs,
    fine_tune_epochs=fine_tune_epochs,
    base_model_name="base_model"
)

In [10]:
tuner = kt.BayesianOptimization(
    hypermodel=hypermodel,
    objective="val_loss",
    max_trials=max_opt_trials,
    seed=seed,
    directory="../model_training/",
    project_name=project_name
)

In [11]:
es_callback = EarlyStopping(monitor=es_criterion, mode=es_mode, min_delta=es_min_delta, patience=es_patience)

In [12]:
tuner.search(
    train_generator,
    validation_data=val_generator,
    # steps_per_epoch=train_generator.n // batch_size,
    # validation_steps=val_generator.n // batch_size,
    callbacks=[es_callback],
    workers=n_cores
)


Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
last_resnet_layer |conv5_block1_out  |?                 
frozen_lr         |0.0015277         |?                 
adam_beta_1       |0.86971           |?                 
adam_beta_2       |0.99798           |?                 

Epoch 1/10
552/552 [==============================] - 36s 52ms/step - loss: 0.2616 - binary_accuracy: 0.8914 - val_loss: 0.2065 - val_binary_accuracy: 0.9199
Epoch 2/10
552/552 [==============================] - 23s 42ms/step - loss: 0.1879 - binary_accuracy: 0.9299 - val_loss: 0.1907 - val_binary_accuracy: 0.9276
Epoch 1/10
552/552 [==============================] - 59s 93ms/step - loss: 0.1310 - binary_accuracy: 0.9484 - val_loss: 0.1415 - val_binary_accuracy: 0.9503
Epoch 2/10
498/552 [==========================>...] - ETA: 4s - loss: 0.1013 - binary_accuracy: 0.9659

KeyboardInterrupt: 

In [ ]:
tuner.results_summary()

In [ ]:
best_hp = tuner.get_best_hyperparameters()[0]

In [ ]:
best_hp.values

In [ ]:
best_tl_model = tuner.get_best_models()[0]

In [ ]:
best_tl_model.summary()